In [1]:
%cd workspace
!git clone -q https://github.com/tloen/alpaca-lora.git
%cd alpaca-lora
!pip install -r requirements.txt -q
!pip uninstall peft -y -q
!pip install -q git+https://github.com/huggingface/peft.git@e536616888d51b453ed354a6f1e243fecb02ea08

/workspace
fatal: destination path 'alpaca-lora' already exists and is not an empty directory.
/workspace/alpaca-lora


In [2]:
!wget -q https://github.com/Beomi/KoAlpaca/raw/main/ko_alpaca_data.json

In [3]:
import json

with open('ko_alpaca_data.json','r') as f:
    dataset = json.load(f)
    
dataset[:5]

[{'instruction': '건강을 유지하기 위한 세 가지 팁을 알려주세요.',
  'input': '',
  'output': '세 가지 팁은 아침식사를 꼭 챙기며, 충분한 수면을 취하고, 적극적으로 운동을 하는 것입니다.'},
 {'instruction': '세 가지 기본 색은 무엇인가요?',
  'input': '',
  'output': '기본 색은 빨강, 파랑, 노랑입니다.'},
 {'instruction': '원자의 구조를 설명하세요.',
  'input': '',
  'output': '원자는 양성자, 중성자, 전자로 구성되어 있으며, 양성자와 중성자는 원자핵 안에 있고 전자는 주변에 있습니다.'},
 {'instruction': '대기 오염을 어떻게 줄일 수 있나요?',
  'input': '',
  'output': '대기 오염을 줄이기 위해 자동차 대신 대중교통을 이용하고, 미세먼지 배출량을 줄이기 위해 차량 정기 검사를 받아야 합니다. 또한 에너지 절약을 위해 전기를 아끼고, 나무를 심어 대기 오염을 줄일 수 있습니다.'},
 {'instruction': '어려운 결정을 내려야 했던 때를 설명하세요.',
  'input': '',
  'output': '제가 어려운 결정을 내려야 했던 때는 대학원 졸업 후 직장을 찾아야 했던 때입니다. 다양한 선택지가 있었지만 나의 관심사와 비전에 가장 부합하는 경력을 뽑는데 어려움이 있었습니다. 이 때 여러 사람들의 조언과 현재 상황에 대한 깊은 고민 끝에 적절한 선택을 내리게 되었습니다.'}]

In [4]:
prompt_template = {
    "description": "Alpaca-LoRA Custom 템플릿",
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context.\n"
        "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n\n"
        "Write a response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Input(입력):\n{input}\n\n### Response:\n"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task.\n"
        "아래는 작업을 설명하는 명령어입니다.\n\n"
        "Write a response that appropriately completes the request.\n명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Response:\n"
    ),
    "response_split": "### Response:", #전체 prompt 와 response 를 구분
}

with open('templates/custom.json', 'w', encoding='utf-8') as f:
    json.dump(prompt_template, f, ensure_ascii=False)

In [4]:
!pip install ipywidgets

In [6]:
from huggingface_hub import login
login()

In [7]:
!python finetune.py \
    --base_model 'baffo32/decapoda-research-llama-7B-hf' \
    --data_path 'ko_alpaca_data.json' \
    --output_dir './output' \
    --num_epochs 5 \
    --learning_rate 5e-4 \
    --val_set_size 2000 \
    --batch_size 512 \
    --micro_batch_size 16 \
    --prompt_template_name 'custom'

Training Alpaca-LoRA model with params:
base_model: baffo32/decapoda-research-llama-7B-hf
data_path: ko_alpaca_data.json
output_dir: ./output
batch_size: 512
micro_batch_size: 16
num_epochs: 5
learning_rate: 0.0005
cutoff_len: 256
val_set_size: 2000
lora_r: 8
lora_alpha: 16
lora_dropout: 0.05
lora_target_modules: ['q_proj', 'v_proj']
train_on_inputs: True
add_eos_token: False
group_by_length: False
wandb_project: 
wandb_run_name: 
wandb_watch: 
wandb_log_model: 
resume_from_checkpoint: False
prompt template: custom

Loading checkpoint shards: 100%|████████████████| 33/33 [00:11<00:00,  2.98it/s]
trainable params: 4194304 || all params: 6742609920 || trainable%: 0.06220594176090199
Map: 100%|█████████████████████████| 2000/2000 [00:01<00:00, 1016.73 examples/s]
Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
{'loss': 1.5176, 'learning_rate': 5e-0

In [4]:
!pip install gradio==3.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 565.5 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: gradio
    Found existing installation: gradio 3.5
    Uninstalling gradio-3.5:
      Successfully uninstalled gradio-3.5


In [5]:
!python generate.py \
    --base_model 'baffo32/decapoda-research-llama-7B-hf' \
    --lora_weights './output' \
    --prompt_template 'custom' \
    --share_gradio

Traceback (most recent call last):
  File "/workspace/alpaca-lora/generate.py", line 5, in <module>
    import gradio as gr
  File "/opt/conda/lib/python3.10/site-packages/gradio/__init__.py", line 3, in <module>
    import gradio.components as components
  File "/opt/conda/lib/python3.10/site-packages/gradio/components.py", line 23, in <module>
    import altair as alt
  File "/opt/conda/lib/python3.10/site-packages/altair/__init__.py", line 620, in <module>
    from .vegalite import *
  File "/opt/conda/lib/python3.10/site-packages/altair/vegalite/__init__.py", line 2, in <module>
    from .v5 import *
  File "/opt/conda/lib/python3.10/site-packages/altair/vegalite/v5/__init__.py", line 2, in <module>
    from .schema import *
  File "/opt/conda/lib/python3.10/site-packages/altair/vegalite/v5/schema/__init__.py", line 3, in <module>
    from .core import *
  File "/opt/conda/lib/python3.10/site-packages/altair/vegalite/v5/schema/core.py", line 7, in <module>
    from altair.utils.sch